In [0]:
from google.colab import drive

In [2]:
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os
os.chdir("/content/drive/My Drive/Colab Notebooks/L7")

In [4]:
!pip install deepctr
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from deepctr.models import WDL
from deepctr.inputs import SparseFeat,get_feature_names

     |████████████████████████████████| 92kB 2.5MB/s 


In [5]:
df = pd.read_csv('./movielens_data/ratings.dat', sep = '::',header = None)
movies = pd.read_csv('./movielens_data/movies.dat', sep = '::',header = None)
users = pd.read_csv('./movielens_data/users.dat', sep = '::',header = None)
df.columns = ['userId','movieId','Rating','timestamp']
movies.columns = ['movieId','name','tag']
users.columns = ['userId','sex','age','occupation','zipcode']
df_tmp = pd.merge(df, movies, on = 'movieId')
data_whole = pd.merge(df_tmp,users, on = 'userId')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  This is separate from the ipykernel 

In [6]:
data_whole.shape

(1000209, 10)

In [0]:
sparse_features = ["movieId", "userId", "sex", "age", "occupation", "zipcode"]
target = ['Rating']

In [0]:
# 对特征标签进行编码
for feature in sparse_features:
    lbe = LabelEncoder()
    data_whole[feature] = lbe.fit_transform(data_whole[feature])

In [0]:
# 计算每个特征中的 不同特征值的个数
fixlen_feature_columns = [SparseFeat(feature, data_whole[feature].nunique()) for feature in sparse_features]
linear_feature_columns = fixlen_feature_columns
dnn_feature_columns = fixlen_feature_columns
feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)

In [0]:
# 将数据集切分成训练集和测试集
train, test = train_test_split(data_whole, test_size=0.2)
train_model_input = {name:train[name].values for name in feature_names}
test_model_input = {name:test[name].values for name in feature_names}

In [17]:
# 使用WDL进行训练
model = WDL(linear_feature_columns, dnn_feature_columns, task='regression')
model.compile("adam", "mse", metrics=['mse'], )
history = model.fit(train_model_input, train[target].values, batch_size=2560, epochs=100, verbose=True, validation_split=0.2 )

Train on 640133 samples, validate on 160034 samples
Epoch 1/100
640133/640133 [==============================] - 5s 7us/sample - loss: 2.2431 - mean_squared_error: 2.2427 - val_loss: 0.8335 - val_mean_squared_error: 0.8328
Epoch 2/100
640133/640133 [==============================] - 4s 7us/sample - loss: 0.8173 - mean_squared_error: 0.8165 - val_loss: 0.8118 - val_mean_squared_error: 0.8109
Epoch 3/100
640133/640133 [==============================] - 4s 7us/sample - loss: 0.7968 - mean_squared_error: 0.7959 - val_loss: 0.8049 - val_mean_squared_error: 0.8039
Epoch 4/100
640133/640133 [==============================] - 4s 7us/sample - loss: 0.7846 - mean_squared_error: 0.7835 - val_loss: 0.7963 - val_mean_squared_error: 0.7952
Epoch 5/100
640133/640133 [==============================] - 4s 7us/sample - loss: 0.7739 - mean_squared_error: 0.7726 - val_loss: 0.7911 - val_mean_squared_error: 0.7898
Epoch 6/100
640133/640133 [==============================] - 4s 7us/sample - loss: 0.7631 - m

In [0]:
# 使用WDL进行预测
pred_ans = model.predict(test_model_input, batch_size=256)

In [0]:
# 输出RMSE或MSE
mse = round(mean_squared_error(test[target].values, pred_ans), 4)

In [0]:
rmse = mse ** 0.5

In [21]:
print("test RMSE", rmse)

test RMSE 0.9078546139112804


In [22]:
print(pred_ans)

[[2.7016916]
 [3.5201604]
 [3.245679 ]
 ...
 [3.70538  ]
 [3.9519126]
 [4.7778325]]
